In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [5]:
STORM_NUMBER = 17
TITLE = "6 February 2021"
SUBTITLE = "8 AM 7 February 2021"
sts = utc(2021, 2, 6, 12)
ets = utc(2021, 2, 7, 14, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [6]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [25]:
cull = ['IA-DL-33', 'IA-DV-14', 'RANI4', 'IA-AP-9', 'ALRI4', 'CHRI4', 'IA-WY-10', 'IA-LC-12', 'IA-WY-7', 'IA-WY-9']
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["val"] < 1].index, USEME] = False
hardcode = [
    ['IA-WB-17', 0.0001],
    ['OAKI4', 2],
    ['UNDI4', 2],
    ['GRTM7', 2],
    ['KNXI4', 1],
    ['PEAI4', 1],
    ['IA-WR-5', 2],
    ['IA-WR-7', 2],
    ['IA-DV-3', 2],
    ['IA-IA-13', 0.9],
    ['WSHI4', 0.9],
    [924, 2],
    ['FRFI4', 1],
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=3, upper=5)
res = mp.postprocess(filename='210207.png')
mp.close()

     state  wfo  val       lon      lat                               geo  \
239     NE  GID  4.9  -98.4300  40.6000    POINT (132638.379 -487876.843)   
240     NE  GID  4.8  -97.9400  40.5000    POINT (174291.688 -498076.106)   
241     CO  BOU  4.8 -105.8900  40.0100   POINT (-501589.419 -536838.288)   
242     VA  RNK  4.8  -79.2200  37.9100   POINT (1803246.924 -561351.445)   
243     MT  TFX  4.8 -111.3300  45.2800     POINT (-882855.119 93135.873)   
...    ...  ...  ...       ...      ...                               ...   
1259    AK  AFG  3.0 -147.6814  64.8972  POINT (-2082313.772 2909832.933)   
1260    WV  RLX  3.0  -81.1833  37.7833   POINT (1639629.762 -615899.982)   
1261    NE  LBF  3.0  -98.7894  41.7989    POINT (100386.775 -355160.159)   
1262    WY  RIW  3.0 -110.9322  42.7306   POINT (-889791.263 -192718.911)   
1263    SD  UNR  3.0 -102.1431  43.4401   POINT (-173020.591 -171174.750)   

      used_for_analysis  nwsli  plotme source  
239                True    